In [103]:
import pandas as pd

In [104]:
#load all ti matches
matches_df = pd.read_json('all_matches.json')
matches_df['match_id'].count()



50

In [105]:
def get_winner(matchid):
    match_data = matches_df[matches_df['match_id']==matchid]
    radiant_win = match_data['radiant_win'].to_string(index=False)
    winner = 1
    if radiant_win == 'True':
        winner = 0
    return winner



In [106]:
#define heroes and teams
heroes_df = pd.read_json('heroes.json')
heroes_df = heroes_df[['id','localized_name']]
heroes_df.rename(columns={"localized_name":"hero_name"},inplace=True)
heroes_df.head()


,id,hero_name
0,1,Anti-Mage
1,2,Axe
2,3,Bane
3,4,Bloodseeker
4,5,Crystal Maiden


In [107]:
#create pickbans df
picks_bans = matches_df.set_index('match_id').to_dict()['picks_bans']
picks = []
bans = []
for i in picks_bans:
    winner = get_winner(i)
    for j in picks_bans[i]:
        if j['is_pick']:
            pick = {}
            pick['match_id'] = i
            pick['order'] = j['order']
            pick['hero_id'] = j['hero_id']
            pick['team'] = j['team']
            if winner == j['team']:
                pick['win'] = 1
            else:
                pick['win'] = 0
            picks.append(pick)
        elif not j['is_pick']:
            ban = {}
            ban['match_id'] = i
            ban['order'] = j['order']
            ban['hero_id'] = j['hero_id']
            ban['team'] = j['team']
            bans.append(ban)
                      




In [108]:
picks_df = pd.DataFrame(picks)
heroes_df['pick_count'] = heroes_df['id'].map(picks_df['hero_id'].value_counts())
wins = picks_df[['hero_id','win']].groupby(['hero_id']).sum()
wins = wins.to_dict()
heroes_df['wins'] = heroes_df['id'].map(wins['win'])
heroes_df['winrate'] = round(heroes_df['wins'] / heroes_df['pick_count'] * 100,2)

bans_df = pd.DataFrame(bans)
heroes_df['ban_count'] = heroes_df['id'].map(bans_df['hero_id'].value_counts())


heroes_df

,id,hero_name,pick_count,wins,winrate,ban_count
0,1,Anti-Mage,1.0,1.0,100.00,NaN
1,2,Axe,NaN,NaN,NaN,2.0
2,3,Bane,1.0,0.0,0.00,NaN
3,4,Bloodseeker,1.0,0.0,0.00,15.0
4,5,Crystal Maiden,7.0,6.0,85.71,NaN
...,...,...,...,...,...,...
118,128,Snapfire,16.0,8.0,50.00,4.0
119,129,Mars,9.0,3.0,33.33,11.0
120,135,Dawnbreaker,5.0,3.0,60.00,4.0
121,136,Marci,18.0,9.0,50.00,31.0


In [109]:
players = matches_df.set_index('match_id').to_dict()['players']
hero_stats = []
for i in players:
    for j in players[i]:
        hero_stat = {}
        hero_stat['match_id'] = j['match_id']
        hero_stat['player_name'] = j['name']
        hero_stat['account_id'] = j['account_id']
        hero_stat['hero_id'] = j['hero_id']
        hero_stat['kills'] = j['kills']
        hero_stat['deaths'] = j['deaths']
        hero_stat['assists'] = j['assists']
        hero_stat['last_hits'] = j['last_hits']
        hero_stat['xp_per_min'] = j['xp_per_min']
        hero_stat['gold_per_min'] = j['gold_per_min']
        hero_stats.append(hero_stat)
hero_stats_df = pd.DataFrame(hero_stats)
hero_stats_df

,match_id,player_name,account_id,hero_id,kills,deaths,assists,last_hits,xp_per_min,gold_per_min
0,6806663153,Fata,86799300,102,0,4,7,51,361,222
1,6806663153,Quinn,221666230,52,6,4,5,300,578,526
2,6806663153,LESLÃO,87063175,137,6,4,6,178,462,378
3,6806663153,Yawar,108452107,10,2,0,1,331,647,535
4,6806663153,MSS,86726887,128,2,6,10,42,301,187
...,...,...,...,...,...,...,...,...,...,...
495,6806767627,Ghost,206642367,80,2,3,13,645,669,623
496,6806767627,Somnus,106863163,52,13,3,4,703,757,777
497,6806767627,Chalice,94738847,77,3,2,12,479,559,560
498,6806767627,kaka,139876032,19,4,8,13,163,495,354


In [110]:
hero_avg_kills = hero_stats_df[['hero_id','kills']].groupby('hero_id').mean()
hero_avg_kills = hero_avg_kills.to_dict()
heroes_df['avg_kills'] = heroes_df['id'].map(hero_avg_kills['kills'])

hero_max_kills = hero_stats_df[['hero_id','kills']].groupby('hero_id').max()
hero_max_kills = hero_max_kills.to_dict()
heroes_df['max_kills'] = heroes_df['id'].map(hero_max_kills['kills'])

hero_avg_deaths = hero_stats_df[['hero_id','deaths']].groupby('hero_id').mean()
hero_avg_deaths = hero_avg_deaths.to_dict()
heroes_df['avg_deaths'] = heroes_df['id'].map(hero_avg_deaths['deaths'])

hero_max_deaths = hero_stats_df[['hero_id','deaths']].groupby('hero_id').max()
hero_max_deaths = hero_max_deaths.to_dict()
heroes_df['max_deaths'] = heroes_df['id'].map(hero_max_deaths['deaths'])

hero_avg_assists = hero_stats_df[['hero_id','assists']].groupby('hero_id').mean()
hero_avg_assists = hero_avg_assists.to_dict()
heroes_df['avg_assists'] = heroes_df['id'].map(hero_avg_assists['assists'])

hero_max_assists = hero_stats_df[['hero_id','assists']].groupby('hero_id').max()
hero_max_assists = hero_max_assists.to_dict()
heroes_df['max_assists'] = heroes_df['id'].map(hero_max_assists['assists'])

hero_avg_last_hits = hero_stats_df[['hero_id','last_hits']].groupby('hero_id').mean()
hero_avg_last_hits = hero_avg_last_hits.to_dict()
heroes_df['avg_last_hits'] = heroes_df['id'].map(hero_avg_last_hits['last_hits'])

hero_max_last_hits = hero_stats_df[['hero_id','last_hits']].groupby('hero_id').max()
hero_max_last_hits = hero_max_last_hits.to_dict()
heroes_df['max_last_hits'] = heroes_df['id'].map(hero_max_last_hits['last_hits'])

hero_avg_xpm = hero_stats_df[['hero_id','xp_per_min']].groupby('hero_id').mean()
hero_avg_xpm = hero_avg_xpm.to_dict()
heroes_df['avg_xp_per_min'] = heroes_df['id'].map(hero_avg_xpm['xp_per_min'])

hero_avg_gpm = hero_stats_df[['hero_id','gold_per_min']].groupby('hero_id').mean()
hero_avg_gpm = hero_avg_gpm.to_dict()
heroes_df['avg_gold_per_min'] = heroes_df['id'].map(hero_avg_gpm['gold_per_min'])

heroes_df

,id,hero_name,pick_count,wins,winrate,ban_count,avg_kills,max_kills,avg_deaths,max_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min
0,1,Anti-Mage,1.0,1.0,100.00,NaN,7.000000,7.0,6.000000,6.0,13.000000,13.0,471.000000,471.0,927.000000,700.000000
1,2,Axe,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Bane,1.0,0.0,0.00,NaN,0.000000,0.0,6.000000,6.0,2.000000,2.0,41.000000,41.0,237.000000,186.000000
3,4,Bloodseeker,1.0,0.0,0.00,15.0,4.000000,4.0,4.000000,4.0,8.000000,8.0,327.000000,327.0,671.000000,565.000000
4,5,Crystal Maiden,7.0,6.0,85.71,NaN,2.142857,3.0,4.000000,8.0,16.285714,24.0,56.571429,94.0,414.714286,306.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,128,Snapfire,16.0,8.0,50.00,4.0,2.687500,6.0,6.000000,12.0,13.750000,23.0,86.312500,203.0,412.250000,298.937500
119,129,Mars,9.0,3.0,33.33,11.0,4.000000,14.0,4.888889,10.0,10.444444,27.0,225.333333,296.0,531.777778,435.666667
120,135,Dawnbreaker,5.0,3.0,60.00,4.0,5.400000,8.0,5.000000,11.0,12.600000,20.0,233.000000,362.0,626.000000,488.000000
121,136,Marci,18.0,9.0,50.00,31.0,3.888889,13.0,6.611111,16.0,11.888889,25.0,92.222222,256.0,458.388889,310.333333


In [111]:
heroes_morepicks_df = heroes_df[heroes_df['pick_count']>=5]

# The most pick Hero is:

In [112]:
hero_picks = heroes_df[['hero_name','pick_count']]
pick_max = hero_picks['pick_count'].max()
hero_picks[hero_picks['pick_count']==pick_max]


,hero_name,pick_count
50,Leshrac,20.0


# The most banned hero is:

In [113]:
ban_picks = heroes_df[['hero_name','ban_count']]
ban_max = ban_picks['ban_count'].max()
ban_picks[ban_picks['ban_count']==ban_max]

,hero_name,ban_count
31,Enigma,35.0


# The highest winrate hero is:

In [114]:
winrate_df = heroes_morepicks_df[['hero_name','winrate']]
win_max = winrate_df['winrate'].max()
winrate_df[winrate_df['winrate']==win_max]

,hero_name,winrate
4,Crystal Maiden,85.71


# The highest kill average hero is:

In [115]:
hero_kill_avg = heroes_morepicks_df[['hero_name','avg_kills']]
avg_kill_max = hero_kill_avg['avg_kills'].max()
hero_kill_avg[hero_kill_avg['avg_kills']==avg_kill_max]



,hero_name,avg_kills
68,Ursa,11.0


# The highest assist average hero is:

In [116]:
hero_asst_avg = heroes_morepicks_df[['hero_name','avg_assists']]
avg_asst_max = hero_asst_avg['avg_assists'].max()
hero_asst_avg[hero_asst_avg['avg_assists']==avg_asst_max]

,hero_name,avg_assists
73,Silencer,17.727273


# The lowest death average hero is:

In [117]:
hero_deaths_avg = heroes_morepicks_df[['hero_name','avg_deaths']]
avg_deaths_min = hero_deaths_avg['avg_deaths'].min()
hero_deaths_avg[hero_deaths_avg['avg_deaths']==avg_deaths_min]

,hero_name,avg_deaths
41,Death Prophet,2.666667
116,Hoodwink,2.666667


# The highest last hit average hero is:

In [118]:
hero_lh_avg = heroes_morepicks_df[['hero_name','avg_last_hits']]
avg_lh_max = hero_lh_avg['avg_last_hits'].max()
hero_lh_avg[hero_lh_avg['avg_last_hits']==avg_lh_max]

,hero_name,avg_last_hits
87,Naga Siren,567.857143


# The highest XPM average hero is:

In [119]:
hero_xpm_avg = heroes_morepicks_df[['hero_name','avg_xp_per_min']]
avg_xpm_max = hero_xpm_avg['avg_xp_per_min'].max()
hero_xpm_avg[hero_xpm_avg['avg_xp_per_min']==avg_xpm_max]

,hero_name,avg_xp_per_min
13,Pudge,762.25


1. Highest Kill Average
2. Highest Assist Average
3. Lowest Death Average
4. Highest Last Hit Average
5. Highest XPM Average
6. Most Kills in a game
7. Most Last Hit in a game

# The most kills in a game hero is:

In [120]:
hero_max_kills = heroes_morepicks_df[['hero_name','max_kills']]
maxkills = hero_max_kills['max_kills'].max()
hero_max_kills[hero_max_kills['max_kills']==maxkills]

,hero_name,max_kills
50,Leshrac,22.0


# The most last hit in a gamer hero is:

In [121]:
hero_max_lh = heroes_morepicks_df[['hero_name','max_last_hits']]
maxlh = hero_max_lh['max_last_hits'].max()
hero_max_lh[hero_max_lh['max_last_hits']==maxlh]

,hero_name,max_last_hits
33,Sniper,1058.0
